### В ноутбуке используется только одна модель - starspace, без стыковки с catboost или autogluon


In [1]:
# import sys
import os
import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import make_scorer, mean_absolute_error, roc_auc_score
from sklearn.pipeline import Pipeline

from sklearn.pipeline import make_union
from sklearn.preprocessing import FunctionTransformer

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from category_encoders import OrdinalEncoder, OneHotEncoder, CountEncoder
import sys
import starwrap as sw
import starspace_resources
from operator import itemgetter



from pathlib import Path
import pickle
import gc


sys.path.append('../')
import config


pd.set_option('display.max_rows', 120)
plt.style.use("dark_background")
plt.rcParams['figure.figsize'] = (20, 15)

os.environ['PYDEVD_USE_CYTHON'] = 'Yes'

print(Path.cwd())


/home/sergey/mnt/st1500/Usr/Sergey/TheJob/Otus/ML_advanced2020/home_works/1. Auto_ML/drivendata_flu_shot/starspace36_mono_model


In [2]:
X = pd.read_csv("../input/training_set_features.csv", index_col='respondent_id', low_memory=False)
y = pd.read_csv("../input/training_set_labels.csv", index_col='respondent_id', low_memory=False)
submission_format = pd.read_csv("../input/submission_format.csv", index_col='respondent_id', low_memory=False)
test_set = pd.read_csv("../input/test_set_features.csv", index_col='respondent_id', low_memory=False)

In [3]:
print('y.head()')
y.head()

y.head()


,h1n1_vaccine,seasonal_vaccine
respondent_id,,
0,0,0
1,0,1
2,0,0
3,0,1
4,0,0


In [4]:
print("test_set")
test_set.head()


test_set


,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [5]:
# split for starspace and catboost datasets
X_starspace = X
y_starspace = y


In [6]:
# train corpus
starspace_keys = starspace_resources.make_train_starspace_corpus(
    X_starspace, y_starspace,
    starspace_path=Path.cwd(),
    to_file=True)

print(starspace_keys)

dict_keys(['h1n1_vaccine', 'seasonal_vaccine', 'both_labels'])


In [7]:
def get_model(starspace_key, force=False):
    arg = sw.args()
    arg.trainFile = f'{starspace_key}.txt'
    arg.trainMode = 0
    arg.epoch = 10
    arg.lr = 0.01
    arg.ngrams = 2

    sp = sw.starSpace(arg)

    model_fname = Path(f'starspace_{starspace_key}_model')
    if (not force) and (model_fname.is_file()):
        result = sp.initFromSavedModel(str(model_fname))
        print('model loaded', model_fname)
    else:
        sp.init()
        sp.train()
        result = sp
        sp.saveModel(str(model_fname))
        print('successfully saved model ', str(model_fname))

    return result

In [8]:
# test corpus
# Preprocess test data
test_corpus_fname = starspace_resources.make_test_corpus(test_set)
test_corpus = pd.read_csv(test_corpus_fname, index_col='respondent_id', low_memory=False)

print('**************   test_corpus.head()  *************************')
test_corpus.head()

**************   test_corpus.head()  *************************


,0
respondent_id,
26707,0_2.0 1_2.0 2_0.0 3_1.0 4_0.0 5_1.0 6_1.0 7_0....
26708,0_1.0 1_1.0 2_0.0 3_0.0 4_0.0 5_0.0 6_0.0 7_0....
26709,0_2.0 1_2.0 2_0.0 3_0.0 4_1.0 5_1.0 6_1.0 7_1....
26710,0_1.0 1_1.0 2_0.0 3_0.0 4_0.0 5_0.0 6_0.0 7_0....
26711,0_3.0 1_1.0 2_1.0 3_1.0 4_0.0 5_1.0 6_1.0 7_1....


In [9]:
print('getting model ..')
model = get_model(starspace_key='h1n1_vaccine', force=True)

print('model got!')
print(model)

getting model ..
successfully saved model  starspace_h1n1_vaccine_model
model got!


In [18]:
count = 0
for ind, body_str in test_corpus.iterrows():
    count+=1
    print(*body_str.values)
    dict_obj = model.predictTags(*body_str.values, 0)
    print(dict_obj)
    dict_obj = sorted(dict_obj.items(), key=itemgetter(1), reverse=True)
    if count > 5: break

print('ok')

0_2.0 1_2.0 2_0.0 3_1.0 4_0.0 5_1.0 6_1.0 7_0.0 8_1.0 9_0.0 10_0.0 11_0.0 12_0.0 13_0.0 14_1.0 15_5.0 16_1.0 17_1.0 18_5.0 19_1.0 20_1.0 21_2 22_4.0 23_Hispanic 24_Female 25_3.0 26_Not Married 27_Rent 28_Employed 29_mlyzmhmf 30_MSA, Not Principle  City 31_1.0 32_0.0 33_atmlpfrs 34_hfxkjkmi
{}
0_1.0 1_1.0 2_0.0 3_0.0 4_0.0 5_0.0 6_0.0 7_0.0 8_0.0 9_0.0 10_0.0 11_0.0 12_0.0 13_0.0 14_0.0 15_4.0 16_1.0 17_1.0 18_4.0 19_1.0 20_1.0 21_1 22_3.0 23_White 24_Male 25_1.0 26_Not Married 27_Rent 28_Employed 29_bhuqouqj 30_Non-MSA 31_3.0 32_0.0 33_atmlpfrs 34_xqwwgdyp
{}
0_2.0 1_2.0 2_0.0 3_0.0 4_1.0 5_1.0 6_1.0 7_1.0 8_1.0 9_0.0 10_0.0 11_0.0 12_0.0 13_0.0 14_nan 15_5.0 16_4.0 17_2.0 18_5.0 19_4.0 20_4.0 21_4 22_4.0 23_White 24_Male 25_3.0 26_Married 27_Own 28_Employed 29_lrircsnp 30_Non-MSA 31_1.0 32_0.0 33_nduyfdeo 34_pvmttkik
{}
0_1.0 1_1.0 2_0.0 3_0.0 4_0.0 5_0.0 6_0.0 7_0.0 8_0.0 9_1.0 10_1.0 11_1.0 12_0.0 13_0.0 14_1.0 15_4.0 16_2.0 17_2.0 18_4.0 19_4.0 20_2.0 21_5 22_3.0 23_White 24_Female